In [61]:
import sys
print(sys.executable)
from azure.cosmos import CosmosClient, PartitionKey, exceptions
from tqdm import tqdm

/Library/Frameworks/Python.framework/Versions/3.6/bin/python3


In [62]:
ACCOUNT_URI = "https://lolpredict.documents.azure.com:443/"
ACCOUNT_KEY = "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ=="
client = CosmosClient(ACCOUNT_URI, credential=ACCOUNT_KEY)


## Create League Database

In [54]:
def create_database(database_name):
    try:
        database = client.create_database(database_name)
    except exceptions.CosmosResourceExistsError:
        database = client.get_database_client(database_name)
    return database

def create_container(database, container_name):
    try:
        container = database.create_container(id=container_name,    partition_key=PartitionKey(path="/league"))
    except exceptions.CosmosResourceExistsError:
        container = database.get_container_client(container_name)
    except exceptions.CosmosHttpResponseError:
        raise
    return container

def get_database(database_name):
    return client.get_database_client(database_name)

def get_container(database, container_name):
    return database.get_container_client(container_name)

def insert_item(container,data):
    container.upsert_item(data)

In [55]:
database = create_database("league_all_data")
champion_container = create_container(database, "champion")


## send champion data to azure 

In [56]:
import json 

In [57]:
count = 0
with open("LOL/league_predict/champion.json","rb") as champ:
    champ_json = json.load(champ)
    #load through each champ

In [60]:
for c in tqdm(champ_json['data'].keys()):
    c_data_raw = champ_json['data'][c]
    champ_dict = {}
    champ_dict['id'] = c_data_raw['key']
    champ_dict['name'] = c
    champ_dict['stats'] = c_data_raw['stats']
    champ_dict['tags'] = c_data_raw['tags']
    insert_item(champion_container,champ_dict)
print("Done insert data to cosmosDB") 

HBox(children=(IntProgress(value=0, max=146), HTML(value='')))


Done insert data to cosmosDB


In [42]:
c_data_raw['key']

## Querying champion data from CosmosDB

In [79]:
query = "SELECT * FROM mycontainer r WHERE r.id='23'"
for item in champion_container.query_items(query=query, enable_cross_partition_query=True):
    print(json.dumps(item, indent=True))
    print("-"*10)

{
 "id": "23",
 "name": "Tryndamere",
 "stats": {
  "hp": 625.64,
  "hpperlevel": 98,
  "mp": 100,
  "mpperlevel": 0,
  "movespeed": 345,
  "armor": 33,
  "armorperlevel": 3.1,
  "spellblock": 32.1,
  "spellblockperlevel": 1.25,
  "attackrange": 125,
  "hpregen": 8.5,
  "hpregenperlevel": 0.9,
  "mpregen": 0,
  "mpregenperlevel": 0,
  "crit": 0,
  "critperlevel": 0,
  "attackdamage": 69,
  "attackdamageperlevel": 3.7,
  "attackspeedperlevel": 2.9,
  "attackspeed": 0.67
 },
 "tags": [
  "Fighter",
  "Assassin"
 ],
 "_rid": "nXoLAIObOW95AAAAAAAAAA==",
 "_self": "dbs/nXoLAA==/colls/nXoLAIObOW8=/docs/nXoLAIObOW95AAAAAAAAAA==/",
 "_etag": "\"0900f494-0000-0400-0000-5dcf33b90000\"",
 "_attachments": "attachments/",
 "_ts": 1573860281
}
----------
